In [57]:
#этот файл для пространственного анализа

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import trimboth
import ast
import geopy
from tqdm import tqdm
import geopy.distance
from tqdm._tqdm_notebook import tqdm_notebook
tqdm.pandas(desc="Example Desc")

In [2]:
import plotly.offline as pyo
import chart_studio.plotly as chspy
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

init_notebook_mode(connected=True)

In [3]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import seaborn as sns; sns.set()
import csv

In [4]:
flats = pd.read_csv('final.csv', index_col=0)

### Count number of adds in a region

In [5]:
spb = flats[(flats.trade_type == 'sale')&\
            ((flats.city == 'Санкт-Петербург'))]
             #|(flats.city == 'Ленинградская область'))]

In [6]:
spb.groupby('trade_type')['_id'].count().sort_values()

trade_type
sale    13257
Name: _id, dtype: int64

In [7]:
spb.cian_id

3386      225953549.0
3389      225953547.0
5351      225955514.0
6883      225957043.0
6914      225957063.0
             ...     
443601    228206970.0
443727    228208201.0
443767    228254567.0
443782    228208323.0
443822    228297740.0
Name: cian_id, Length: 13257, dtype: float64

Введем расстояние от центра города в spb

In [8]:
def get_dist(x):
    if (np.isnan(x.latitude)  or np.isnan(x.longitude)):
        return np.nan
    else:
        return geopy.distance.vincenty((x.latitude, x.longitude),\
                                     (59.939095, 30.315868)).km

In [9]:
spb['distance'] = spb.progress_apply(get_dist, axis=1)

Example Desc: 100%|████████████████████████████████████████████████████████████| 13257/13257 [00:01<00:00, 7356.85it/s]


In [10]:
spb = spb[spb['distance']<20]

In [11]:
spb.groupby('New_building')['id'].count().sort_values()

New_building
1.0    1657
0.0    6580
Name: id, dtype: int64

In [12]:
spb_cluster = spb[['longitude', 'latitude', 'price_per_sq_meter']]

In [13]:
# from IPython.display import Image
# from IPython.core.display import HTML , display
# x = flats[flats.cian_id==228193926].pic_urls.values[0]#[247768]
# for i in ast.literal_eval(x):
#     display(Image(url= i))

## Statistic analysis

In [14]:
clustering_data=spb[spb.trade_type=='sale']\
[['price_per_sq_meter', 'latitude', 'longitude', '_id']].dropna()

In [15]:
clustering_data.head()

,price_per_sq_meter,latitude,longitude,_id
3386,115713.0,60.023989,30.235693,ObjectId(5e57a1a4089b1f8c93fac038)
3389,78661.0,59.966607,30.480724,ObjectId(5e57a1a9550a1f3209fac030)
5351,31475.0,59.987677,30.333652,ObjectId(5e57bf52eca493834fe53abc)
6883,80986.0,59.956756,30.434947,ObjectId(5e57d681d5f1676340dd8a1b)
6914,100000.0,59.827480,30.168223,ObjectId(5e57d6e4aa71187686dd8a21)


In [16]:
x = clustering_data.price_per_sq_meter
clustering_data.price_per_sq_meter = (x - x.mean())/x.std()/6
x = clustering_data.latitude
clustering_data.latitude = (x - x.mean())/x.std()
x = clustering_data.longitude
clustering_data.longitude = (x - x.mean())/x.std()

In [17]:
kmeans = KMeans(n_clusters = 24, init ='k-means++')
kmeans.fit(clustering_data[['price_per_sq_meter', 'latitude', 'longitude']]) # Compute k-means clustering.
labels = kmeans.predict(clustering_data[['price_per_sq_meter', 'latitude', 'longitude']]) # Labels of each point
clustering_data['cluster_label'] = kmeans.fit_predict(clustering_data\
                                                      [['price_per_sq_meter', 'latitude', 'longitude']])
centers = kmeans.cluster_centers_ # Coordinates of cluster centers.
# clustering_data.head(10)

In [18]:
clustering_data = clustering_data[['_id', 'cluster_label']]
spb = spb.merge(clustering_data, left_on='_id', right_on='_id')

In [19]:
clustering_data.groupby('cluster_label')['_id'].count().sort_values(ascending=False)

cluster_label
0     662
20    536
8     474
15    453
1     448
19    427
11    426
12    401
9     395
5     394
4     384
13    379
6     349
2     322
21    320
10    245
16    233
18    230
14    228
3     223
22    212
7     200
17    156
23    140
Name: _id, dtype: int64

In [20]:
spb['cluster_label'] = spb['cluster_label'].progress_apply(lambda x: str(x))

Example Desc: 100%|████████████████████████████████████████████████████████████| 8237/8237 [00:00<00:00, 433508.78it/s]


Выясняем, какие кластеры целиком лежат в пределах 50 км от центра города и работаем с ними

In [21]:
dct = spb.groupby('cluster_label')['distance'].min().sort_values(ascending=False)
dct = dct.to_dict()

In [22]:
dct

{'7': 12.729918022686501,
 '14': 12.312276667473615,
 '0': 12.247560034194935,
 '12': 10.784569752905591,
 '21': 10.393440632072448,
 '11': 10.393108136657686,
 '20': 9.579072156716812,
 '17': 8.844446157883572,
 '8': 8.028910708278964,
 '6': 7.321032987917042,
 '16': 7.297785213516354,
 '10': 6.996135550419359,
 '18': 6.939857094240596,
 '2': 6.702012180818785,
 '5': 6.543054731865354,
 '13': 6.435778230261539,
 '22': 5.048409078847728,
 '3': 4.764157699105531,
 '1': 3.4480146859222662,
 '4': 2.635724836416143,
 '19': 1.282937670481127,
 '23': 0.4645686480811279,
 '15': 0.4403192030907109,
 '9': 0.4268804921561881}

In [25]:
lst = spb.cluster_label.value_counts().keys()

In [26]:
lst

Index(['0', '20', '8', '15', '1', '19', '11', '12', '9', '5', '4', '13', '6',
       '2', '21', '10', '16', '18', '14', '3', '22', '7', '17', '23'],
      dtype='object')

Visualizating

In [27]:
color_list = px.colors.qualitative.Dark24

In [28]:
fig = make_subplots(
    rows=1, cols=2, subplot_titles=('Map', 'Box plots'),
    specs=[[{"type": "mapbox"}, {"type": "box"}]])

Это на случай если хочется посмотреть новостройки

In [29]:
# data = list()
# #set the geo=spatial data
# data_old = [go.Scattermapbox(
#             lat= spb_old[spb_old["cluster_label"]==lst[i]]['latitude'] ,
#             lon= spb_old[spb_old["cluster_label"]==lst[i]]['longitude'],
#             customdata = spb_old[spb_old["cluster_label"]==lst[i]]['total_price'],
#             mode='markers',
#             marker=dict(
#                 size= 4,
#                 color=color_list[i],
#                 opacity = .8,
#                 symbol = 'circle'
#             ),
#     text=list(spb_old[spb_old["cluster_label"]==lst[i]].Number_of_rooms),
#           ) for i in range(len(lst))]
# data_new = [
# go.Scattermapbox(
#             lat= spb_new[spb_new["cluster_label"]==lst[i]]['latitude'] ,
#             lon= spb_new[spb_new["cluster_label"]==lst[i]]['longitude'],
#             customdata = spb_new[spb_new["cluster_label"]==lst[i]]['total_price'],
#             mode='markers',
#             marker=dict(
#                 size= 4,
#                 opacity = .8,
#                 reversescale = True,
#             autocolorscale = False,
#             symbol = 'square',
#                 color=color_list[i],
#             ),
#     text=list(spb_new[spb_new["cluster_label"]==lst[i]].Number_of_rooms),
#           ) for i in range(len(lst))]
# data.extend(data_old)
# data.extend(data_new)

In [30]:
fig = make_subplots(
    rows=1, cols=2, subplot_titles=('Map', 'Box plots'),
    specs=[[{"type": "mapbox"}, {"type": "Box"}]])

In [31]:
data = list()
#set the geo=spatial data
data = [go.Scattermapbox(
            lat= spb[spb["cluster_label"]==lst[i]]['latitude'] ,
            lon= spb[spb["cluster_label"]==lst[i]]['longitude'],
            customdata = spb[spb["cluster_label"]==lst[i]]['price_per_sq_meter'],
            mode='markers',
            marker=dict(
                size= 4,
                color=color_list[i],
                opacity = .8,
                symbol = 'circle'
            ),
    text=list(spb[spb["cluster_label"]==lst[i]].Number_of_rooms),
    name= lst[i]
          ) for i in range(len(lst))]
for i in data:
    fig.add_trace(i, row=1, col=1)
data = list()
for i in range(len(lst)):
    data.append(
    go.Box(
y = spb.price_per_sq_meter[spb.cluster_label==lst[i]].dropna(),
name=lst[i], marker_color = color_list[i]))
for i in data:
    fig.add_trace(i, row=1, col=2)

In [32]:
fig.update_mapboxes(
        # here you need the token from Mapbox
        accesstoken = 'pk.eyJ1Ijoibmlrc29iOTciLCJhIjoiY2s4cGs3NzRnMDAxODNnbnR0M3QxNHl5NyJ9.Ppwf_rp0v7AP5-VepAWydw',
        bearing = 0,
        # where we want the map to be centered
        center = dict(
            lat = 59.939095,
            lon = 30.315868
        ),
        # we want the map to be "parallel" to our screen, with no angle
        pitch = 0,
        # default level of zoom
        zoom = 8,
        # default map style
        style = "open-street-map")
fig.update_layout(height=650, width=1300, title_text="Whole flats price per meter box plot" );

In [33]:
pyo.plot(fig, filename='visualization/total_map', auto_open=False)

'visualization/total_map.html'

In [34]:
studio_flat = spb[spb.Number_of_rooms==0.8]
# studio_flat.total_area = studio_flat.total_price/studio_flat.price_per_sq_meter
one_flat = spb[spb.Number_of_rooms == 1]
two_flat = spb[spb.Number_of_rooms == 2]
three_flat = spb[spb.Number_of_rooms == 3]
four_flat = spb[spb.Number_of_rooms == 4]
five_flat = spb[spb.Number_of_rooms == 5]
flat_list = [studio_flat, one_flat, two_flat, three_flat, four_flat, five_flat]

In [35]:
new_flt = list()
for flt in flat_list:
    flt = flt[['total_price', 'total_area', \
               'price_per_sq_meter', 'New_building', 'trade_type', 'cluster_label',\
               'distance', 'latitude', 'longitude']]\
    .dropna()[flt.trade_type=='sale']
    flt = flt.loc[flt['total_price'].isin(trimboth(flt.total_price, 0.1))\
                  & flt['total_area'].isin(trimboth(flt.total_area, 0.1))]
    new_flt.append(flt)
#     print(flt.price_per_sq_meter.mean())
studio_flat, one_flat, two_flat, three_flat, four_flat, five_flat = new_flt

# Prices_per_square

## One room flats price per meter box plot

In [36]:
fig = make_subplots(
    rows=1, cols=2, subplot_titles=('Map', 'Box plots'),
    specs=[[{"type": "mapbox"}, {"type": "Box"}]])

In [37]:
data = list()
#set the geo=spatial data
data = [go.Scattermapbox(
            lat= one_flat[one_flat["cluster_label"]==lst[i]]['latitude'] ,
            lon= one_flat[one_flat["cluster_label"]==lst[i]]['longitude'],
            customdata = one_flat[one_flat["cluster_label"]==lst[i]]['price_per_sq_meter'],
            mode='markers',
            marker=dict(
                size= 4,
                color=color_list[i],
                opacity = .8,
                symbol = 'circle'
            ),
    text= list(one_flat[one_flat["cluster_label"]==lst[i]]['price_per_sq_meter']),
    name= lst[i]
          ) for i in range(len(lst))]
for i in data:
    fig.add_trace(i, row=1, col=1)
data = list()
for i in range(len(lst)):
    data.append(
    go.Box(
y = one_flat.price_per_sq_meter[one_flat.cluster_label==lst[i]].dropna(),
name=lst[i], marker_color = color_list[i]))
for i in data:
    fig.add_trace(i, row=1, col=2)

In [38]:
fig.update_mapboxes(
        # here you need the token from Mapbox
        accesstoken = 'pk.eyJ1Ijoibmlrc29iOTciLCJhIjoiY2s4cGs3NzRnMDAxODNnbnR0M3QxNHl5NyJ9.Ppwf_rp0v7AP5-VepAWydw',
        bearing = 0,
        # where we want the map to be centered
        center = dict(
            lat = 59.939095,
            lon = 30.315868
        ),
        # we want the map to be "parallel" to our screen, with no angle
        pitch = 0,
        # default level of zoom
        zoom = 8,
        # default map style
        style = "open-street-map")
fig.update_layout(height=650, width=1300, title_text="Whole flats price per meter box plot" );

In [39]:
pyo.plot(fig, filename='visualization/one_room_price_per_meter_box_plot', auto_open=False)

'visualization/one_room_price_per_meter_box_plot.html'

## Two rooms flats price per meter box plot

In [40]:
fig = make_subplots(
    rows=1, cols=2, subplot_titles=('Map', 'Box plots'),
    specs=[[{"type": "mapbox"}, {"type": "Box"}]])

In [41]:
data = list()
#set the geo=spatial data
data = [go.Scattermapbox(
            lat= two_flat[two_flat["cluster_label"]==lst[i]]['latitude'] ,
            lon= two_flat[two_flat["cluster_label"]==lst[i]]['longitude'],
            customdata = two_flat[two_flat["cluster_label"]==lst[i]]['price_per_sq_meter'],
            mode='markers',
            marker=dict(
                size= 4,
                color=color_list[i],
                opacity = .8,
                symbol = 'circle'
            ),
    text= list(two_flat[two_flat["cluster_label"]==lst[i]]['price_per_sq_meter']),
    name= lst[i]
          ) for i in range(len(lst))]
for i in data:
    fig.add_trace(i, row=1, col=1)
data = list()
for i in range(len(lst)):
    data.append(
    go.Box(
y = two_flat.price_per_sq_meter[two_flat.cluster_label==lst[i]].dropna(),
name=lst[i], marker_color = color_list[i]))
for i in data:
    fig.add_trace(i, row=1, col=2)

In [42]:
fig.update_mapboxes(
        # here you need the token from Mapbox
        accesstoken = 'pk.eyJ1Ijoibmlrc29iOTciLCJhIjoiY2s4cGs3NzRnMDAxODNnbnR0M3QxNHl5NyJ9.Ppwf_rp0v7AP5-VepAWydw',
        bearing = 0,
        # where we want the map to be centered
        center = dict(
            lat = 59.939095,
            lon = 30.315868
        ),
        # we want the map to be "parallel" to our screen, with no angle
        pitch = 0,
        # default level of zoom
        zoom = 8,
        # default map style
        style = "open-street-map")
fig.update_layout(height=650, width=1300, title_text="Whole flats price per meter box plot" );

In [43]:
pyo.plot(fig, filename='visualization/two_rooms_price_per_meter_box_plot', auto_open=False)

'visualization/two_rooms_price_per_meter_box_plot.html'

## Three rooms flats price per meter box plot

In [44]:
fig = make_subplots(
    rows=1, cols=2, subplot_titles=('Map', 'Box plots'),
    specs=[[{"type": "mapbox"}, {"type": "Box"}]])

In [45]:
data = list()
#set the geo=spatial data
data = [go.Scattermapbox(
            lat= three_flat[three_flat["cluster_label"]==lst[i]]['latitude'] ,
            lon= three_flat[three_flat["cluster_label"]==lst[i]]['longitude'],
            customdata = three_flat[three_flat["cluster_label"]==lst[i]]['price_per_sq_meter'],
            mode='markers',
            marker=dict(
                size= 4,
                color=color_list[i],
                opacity = .8,
                symbol = 'circle'
            ),
    text= list(three_flat[three_flat["cluster_label"]==lst[i]]['price_per_sq_meter']),
    name= lst[i]
          ) for i in range(len(lst))]
for i in data:
    fig.add_trace(i, row=1, col=1)
data = list()
for i in range(len(lst)):
    data.append(
    go.Box(
y = three_flat.price_per_sq_meter[three_flat.cluster_label==lst[i]].dropna(),
name=lst[i], marker_color = color_list[i]))
for i in data:
    fig.add_trace(i, row=1, col=2)

In [46]:
fig.update_mapboxes(
        # here you need the token from Mapbox
        accesstoken = 'pk.eyJ1Ijoibmlrc29iOTciLCJhIjoiY2s4cGs3NzRnMDAxODNnbnR0M3QxNHl5NyJ9.Ppwf_rp0v7AP5-VepAWydw',
        bearing = 0,
        # where we want the map to be centered
        center = dict(
            lat = 59.939095,
            lon = 30.315868
        ),
        # we want the map to be "parallel" to our screen, with no angle
        pitch = 0,
        # default level of zoom
        zoom = 8,
        # default map style
        style = "open-street-map")
fig.update_layout(height=650, width=1300, title_text="Whole flats price per meter box plot" );

In [47]:
pyo.plot(fig, filename='visualization/three_rooms_price_per_meter_box_plot', auto_open=False)

'visualization/three_rooms_price_per_meter_box_plot.html'

## Four rooms flats price per meter box plot

In [48]:
fig = make_subplots(
    rows=1, cols=2, subplot_titles=('Map', 'Box plots'),
    specs=[[{"type": "mapbox"}, {"type": "Box"}]])

In [49]:
data = list()
#set the geo=spatial data
data = [go.Scattermapbox(
            lat= four_flat[four_flat["cluster_label"]==lst[i]]['latitude'] ,
            lon= four_flat[four_flat["cluster_label"]==lst[i]]['longitude'],
            customdata = four_flat[four_flat["cluster_label"]==lst[i]]['price_per_sq_meter'],
            mode='markers',
            marker=dict(
                size= 4,
                color=color_list[i],
                opacity = .8,
                symbol = 'circle'
            ),
    text= list(four_flat[four_flat["cluster_label"]==lst[i]]['price_per_sq_meter']),
    name= lst[i]
          ) for i in range(len(lst))]
for i in data:
    fig.add_trace(i, row=1, col=1)
data = list()
for i in range(len(lst)):
    data.append(
    go.Box(
y = four_flat.price_per_sq_meter[four_flat.cluster_label==lst[i]].dropna(),
name=lst[i], marker_color = color_list[i]))
for i in data:
    fig.add_trace(i, row=1, col=2)

In [50]:
fig.update_mapboxes(
        # here you need the token from Mapbox
        accesstoken = 'pk.eyJ1Ijoibmlrc29iOTciLCJhIjoiY2s4cGs3NzRnMDAxODNnbnR0M3QxNHl5NyJ9.Ppwf_rp0v7AP5-VepAWydw',
        bearing = 0,
        # where we want the map to be centered
        center = dict(
            lat = 59.939095,
            lon = 30.315868
        ),
        # we want the map to be "parallel" to our screen, with no angle
        pitch = 0,
        # default level of zoom
        zoom = 8,
        # default map style
        style = "open-street-map")
fig.update_layout(height=650, width=1300, title_text="Whole flats price per meter box plot" );

In [51]:
pyo.plot(fig, filename='visualization/four_rooms_price_per_meter_box_plot', auto_open=False)

'visualization/four_rooms_price_per_meter_box_plot.html'

## Five rooms flats price per meter box plot

In [52]:
fig = make_subplots(
    rows=1, cols=2, subplot_titles=('Map', 'Box plots'),
    specs=[[{"type": "mapbox"}, {"type": "Box"}]])

In [53]:
data = list()
#set the geo=spatial data
data = [go.Scattermapbox(
            lat= five_flat[five_flat["cluster_label"]==lst[i]]['latitude'] ,
            lon= five_flat[five_flat["cluster_label"]==lst[i]]['longitude'],
            customdata = five_flat[five_flat["cluster_label"]==lst[i]]['price_per_sq_meter'],
            mode='markers',
            marker=dict(
                size= 4,
                color=color_list[i],
                opacity = .8,
                symbol = 'circle'
            ),
    text= list(five_flat[five_flat["cluster_label"]==lst[i]]['price_per_sq_meter']),
    name= lst[i]
          ) for i in range(len(lst))]
for i in data:
    fig.add_trace(i, row=1, col=1)
data = list()
for i in range(len(lst)):
    data.append(
    go.Box(
y = five_flat.price_per_sq_meter[five_flat.cluster_label==lst[i]].dropna(),
name=lst[i], marker_color = color_list[i]))
for i in data:
    fig.add_trace(i, row=1, col=2)

In [54]:
fig.update_mapboxes(
        # here you need the token from Mapbox
        accesstoken = 'pk.eyJ1Ijoibmlrc29iOTciLCJhIjoiY2s4cGs3NzRnMDAxODNnbnR0M3QxNHl5NyJ9.Ppwf_rp0v7AP5-VepAWydw',
        bearing = 0,
        # where we want the map to be centered
        center = dict(
            lat = 59.939095,
            lon = 30.315868
        ),
        # we want the map to be "parallel" to our screen, with no angle
        pitch = 0,
        # default level of zoom
        zoom = 8,
        # default map style
        style = "open-street-map")
fig.update_layout(height=650, width=1300, title_text="Whole flats price per meter box plot" );

In [55]:
pyo.plot(fig, filename='visualization/five_rooms_price_per_meter_box_plot', auto_open=False)

'visualization/five_rooms_price_per_meter_box_plot.html'

In [56]:
fig = px.histogram(one_flat[one_flat.cluster_label=='14'], x="price_per_sq_meter"
                   , nbins=20, color="New_building", opacity=0.8)
fig.show()